In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle

In [22]:
df = pd.read_csv('benchpress_angles.csv')
# Filter out the unwanted labels
df = df[~df['label'].isin(['b_excessive_arch_down', 'b_excessive_arch_up'])].reset_index(drop=True)


In [23]:
X = df.drop('label', axis=1)
y = df['label']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [25]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [26]:
fit_models = {}
for algorithm, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algorithm] = model

In [27]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(max_iter=1000))]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [28]:
from sklearn.metrics import classification_report


predictions = {}


for algorithm, model in fit_models.items():
    y_pred = model.predict(X_test)  
    predictions[algorithm] = y_pred  


for algorithm, y_pred in predictions.items():
    print(f'--- {algorithm} ')
    print(classification_report(y_test, y_pred))
    print()

--- lr 
                    precision    recall  f1-score   support

b_arms_spread_down       0.52      0.74      0.61        54
  b_arms_spread_up       0.47      0.47      0.47        47
    b_correct_down       0.50      0.38      0.43        45
      b_correct_up       0.37      0.26      0.30        39

          accuracy                           0.48       185
         macro avg       0.46      0.46      0.45       185
      weighted avg       0.47      0.48      0.47       185


--- rc 
                    precision    recall  f1-score   support

b_arms_spread_down       0.47      0.78      0.58        54
  b_arms_spread_up       0.39      0.28      0.33        47
    b_correct_down       0.54      0.44      0.49        45
      b_correct_up       0.32      0.21      0.25        39

          accuracy                           0.45       185
         macro avg       0.43      0.43      0.41       185
      weighted avg       0.44      0.45      0.42       185


--- rf 
        

In [29]:
metrics = {
    'accuracy': {},
    'precision': {},
    'recall': {},
    'f1-score': {}
}

for algorithm, y_pred in predictions.items():
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    metrics['accuracy'][algorithm] = accuracy
    metrics['precision'][algorithm] = precision
    metrics['recall'][algorithm] = recall
    metrics['f1-score'][algorithm] = f1

for metric, values in metrics.items():
    print(f'--- {metric} ---')
    for algorithm, score in values.items():
        print(f'{algorithm}: {score:.4f}')

--- accuracy ---
lr: 0.4811
rc: 0.4486
rf: 0.7081
gb: 0.6811
--- precision ---
lr: 0.4703
rc: 0.4352
rf: 0.7103
gb: 0.6810
--- recall ---
lr: 0.4811
rc: 0.4486
rf: 0.7081
gb: 0.6811
--- f1-score ---
lr: 0.4657
rc: 0.4242
rf: 0.7079
gb: 0.6795
